In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os

# Python'ın utils modülünü bulabilmesi için proje kök dizinini sys.path'e ekle
PROJECT_ROOT = '/content/drive/MyDrive/phishing_detector_app'
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Gerekli Flask ve Sistem Kütüphaneleri
# app.py'nin en başına veya ilk importların altına ekle
!pip uninstall flask-ngrok -y
!pip install pyngrok # pyngrok'u yükle
!sudo apt-get update && sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-tur libtesseract-dev libleptonica-dev pkg-config
!pip install flask-cors
!pip install pytesseract opencv-python

from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import pickle
import pandas as pd
import numpy as np
from PIL import Image
import pytesseract
import cv2
from io import BytesIO
# os zaten import edildi

from pyngrok import ngrok # BU SATIRI EKLE! (ngrok objesini import etmek için)

# Custom preprocessing ve model sınıflarını import et
from utils.preprocessing import clean_text, extract_text_features # create_dnn_model
# Scikit-learn ve Keras/TF bileşenleri
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
# DNN kullanıyorsan gerekli importlar:
# from scikeras.wrappers import KerasClassifier
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.models import Model as KerasModel
# from tensorflow.keras.layers import Input, Dense, Dropout

# =========================================================================
# GLOBAL DEĞİŞKENLER VE YÜKLEMELER
# =========================================================================

model_classifier = None
loaded_tfidf_vectorizer = None
loaded_scaler = None

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

app = Flask(__name__)
CORS(app)

def load_models():
    global model_classifier, loaded_tfidf_vectorizer, loaded_scaler
    try:
        BASE_DRIVE_PATH = '/content/drive/MyDrive/phishing_detector_app'
        MODELS_DIR = os.path.join(BASE_DRIVE_PATH, 'models')
        MODEL_NAME = "best_phishing_detector_Original_Imbalanced_Logistic_Regression.pkl"

        MODEL_PATH = os.path.join(MODELS_DIR, MODEL_NAME)
        TFIDF_VECTORIZER_PATH = os.path.join(MODELS_DIR, 'tfidf_vectorizer.pkl')
        SCALER_PATH = os.path.join(MODELS_DIR, 'scaler.pkl')

        with open(MODEL_PATH, 'rb') as f:
            model_classifier = pickle.load(f)
        with open(TFIDF_VECTORIZER_PATH, 'rb') as f:
            loaded_tfidf_vectorizer = pickle.load(f)
        with open(SCALER_PATH, 'rb') as f:
            loaded_scaler = pickle.load(f)
        print(f"✅ Tüm modeller ve ön işlemciler başarıyla yüklendi.")
    except Exception as e:
        print(f"❌ Model veya ön işlemciler yüklenirken hata oluştu: {e}")

with app.app_context():
    load_models()

# =========================================================================
# TAHMİN FONKSİYONLARI VE API UÇ NOKTALARI
# =========================================================================

def predict_email_content(email_subject, email_body, email_sender, original_urls_val=0):
    if model_classifier is None or loaded_tfidf_vectorizer is None or loaded_scaler is None:
        return {"prediction": "Hata", "confidence": 0.0, "risk_score": 1.0, "error": "Model veya ön işlemciler başlatılamadı."}

    input_df_raw = pd.DataFrame([{
        'sender': email_sender,
        'receiver': '', 'date': '', 'subject': email_subject, 'body': email_body,
        'label': 0, 'urls': original_urls_val
    }])

    input_df_raw['sender_clean'] = input_df_raw['sender'].apply(clean_text)
    input_df_raw['subject_clean'] = input_df_raw['subject'].apply(clean_text)
    input_df_raw['body_clean'] = input_df_raw['body'].apply(clean_text)
    input_df_raw['combined_text'] = input_df_raw['sender_clean'] + ' ' + input_df_raw['subject_clean'] + ' ' + input_df_raw['body_clean']

    input_text_features = extract_text_features(input_df_raw)
    input_tfidf_features = loaded_tfidf_vectorizer.transform(input_df_raw['combined_text']).toarray()
    input_all_features_combined = np.hstack([input_text_features.values, input_tfidf_features])
    input_scaled_features = loaded_scaler.transform(input_all_features_combined)

    try:
        y_pred_proba = model_classifier.predict_proba(input_scaled_features)
        if y_pred_proba.ndim > 1 and y_pred_proba.shape[1] > 1:
            y_pred_proba_positive_class = y_pred_proba[:, 1]
        else:
            y_pred_proba_positive_class = y_pred_proba.flatten()

        y_pred = (y_pred_proba_positive_class > 0.5).astype(int)

        result = {
            "prediction": "Phishing" if y_pred[0] == 1 else "Normal",
            "confidence": float(y_pred_proba_positive_class[0]),
            "risk_score": float(y_pred_proba_positive_class[0])
        }
        return result
    except Exception as e:
        return {"prediction": "Hata", "confidence": 0.0, "risk_score": 1.0, "error": f"Tahmin sırasında hata oluştu: {e}"}

# API Uç Noktaları
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.is_json:
        data = request.get_json()
        email_subject = data.get('subject', '')
        email_body = data.get('body', '')
        email_sender = data.get('sender', '')

        if not email_body and not email_subject:
            return jsonify({"error": "Metin girişi için 'body' veya 'subject' alanı boş bırakılamaz."}), 400

        result = predict_email_content(email_subject, email_body, email_sender)
        return jsonify(result)

    elif 'file' in request.files:
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "Dosya seçilmedi."}), 400

        # OCR ile metin çıkarma
        try:
            img = Image.open(BytesIO(file.read()))
            img_cv = np.array(img.convert('RGB'))
            img_gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)

            extracted_text = pytesseract.image_to_string(img_gray, lang='eng+tur')

            result = predict_email_content(email_subject="Görselden Çıkarıldı", email_body=extracted_text, email_sender="bilinmeyen@gorsel.com")
            return jsonify(result)

        except Exception as e:
            return jsonify({"error": f"Görsel işleme veya OCR hatası: {e}"}), 500

    return jsonify({"error": "Geçersiz istek. Lütfen JSON verisi veya dosya yükleyin."}), 400

# =========================================================================
# Flask uygulamasını başlat (pyngrok ile tünelleyerek)
# =========================================================================
if __name__ == '__main__':
    # pyngrok'u aktif etmek için bir Ngrok Auth Token'a ihtiyacınız olabilir.
    # Ngrok web sitesinden (ngrok.com) bir token alın ve buraya yapıştırın.
    ngrok.set_auth_token("2xbU50LBIAiPX9TO6WOShOSLJmv_2X3gwK2Tg4hE18Sp15BTF") # Kendi token'ınızı buraya yapıştırın

    # Ngrok tünelini başlat
    # `connect()` metoduna Flask uygulamasının çalıştığı portu (5000) veriyoruz.
    public_url = ngrok.connect(5000)
    print(f"✨ Flask Uygulaması şu adreste çalışıyor: {public_url}")

    # Flask uygulamasını başlat
    app.run(debug=True, host='0.0.0.0', port=5000) # host 0.0.0.0 Colab'da zorunlu olabilir

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe a

ModuleNotFoundError: No module named 'pytesseract'

In [ ]:
!pip uninstall flask-ngrok -y
!pip install pyngrok flask-cors pytesseract opencv-python
!sudo apt-get update && sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-tur libtesseract-dev libleptonica-dev pkg-config

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Get:12 ht

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=b17b91d2d7c3ea8ff1f197b2aa05a22be17e163caffe166811fcbfe85078a57f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
import sys
import os
import threading
import time

# Python'ın utils modülünü bulabilmesi için proje kök dizinini sys.path'e ekle
PROJECT_ROOT = '/content/drive/MyDrive/phishing_detector_app'
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import pickle
import pandas as pd
import numpy as np
from PIL import Image
import pytesseract
import cv2
from io import BytesIO
from pyngrok import ngrok

# Custom preprocessing ve model sınıflarını import et
from utils.preprocessing import clean_text, extract_text_features



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# =========================================================================
# GLOBAL DEĞİŞKENLER VE YÜKLEMELER
# =========================================================================

model_classifier = None
loaded_tfidf_vectorizer = None
loaded_scaler = None

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
TEMPLATES_DIR = os.path.join(PROJECT_ROOT, 'templates')
app = Flask(__name__,
           template_folder=TEMPLATES_DIR)
CORS(app, resources={
    r"/*": {
        "origins": ["chrome-extension://*"],
        "methods": ["GET", "POST", "OPTIONS"],
        "allow_headers": ["Content-Type", "Authorization"]
    }
})

def load_models():
    global model_classifier, loaded_tfidf_vectorizer, loaded_scaler
    try:
        BASE_DRIVE_PATH = '/content/drive/MyDrive/phishing_detector_app'
        MODELS_DIR = os.path.join(BASE_DRIVE_PATH, 'models')
        MODEL_NAME = "best_phishing_detector_Original_Imbalanced_Logistic_Regression.pkl"

        MODEL_PATH = os.path.join(MODELS_DIR, MODEL_NAME)
        TFIDF_VECTORIZER_PATH = os.path.join(MODELS_DIR, 'tfidf_vectorizer.pkl')
        SCALER_PATH = os.path.join(MODELS_DIR, 'scaler.pkl')

        with open(MODEL_PATH, 'rb') as f:
            model_classifier = pickle.load(f)
        with open(TFIDF_VECTORIZER_PATH, 'rb') as f:
            loaded_tfidf_vectorizer = pickle.load(f)
        with open(SCALER_PATH, 'rb') as f:
            loaded_scaler = pickle.load(f)
        print(f"✅ Tüm modeller ve ön işlemciler başarıyla yüklendi.")
        return True
    except Exception as e:
        print(f"❌ Model veya ön işlemciler yüklenirken hata oluştu: {e}")
        return False



In [ ]:
# =========================================================================
# TAHMİN FONKSİYONLARI VE API UÇ NOKTALARI
# =========================================================================

def predict_email_content(email_subject, email_body, email_sender, original_urls_val=0):
    if model_classifier is None or loaded_tfidf_vectorizer is None or loaded_scaler is None:
        return {"prediction": "Hata", "confidence": 0.0, "risk_score": 1.0, "error": "Model veya ön işlemciler başlatılamadı."}

    input_df_raw = pd.DataFrame([{
        'sender': email_sender, 'receiver': '', 'date': '', 'subject': email_subject, 'body': email_body,
        'label': 0, 'urls': original_urls_val
    }])

    input_df_raw['sender_clean'] = input_df_raw['sender'].apply(clean_text)
    input_df_raw['subject_clean'] = input_df_raw['subject'].apply(clean_text)
    input_df_raw['body_clean'] = input_df_raw['body'].apply(clean_text)
    input_df_raw['combined_text'] = input_df_raw['sender_clean'] + ' ' + input_df_raw['subject_clean'] + ' ' + input_df_raw['body_clean']

    input_text_features = extract_text_features(input_df_raw)
    input_tfidf_features = loaded_tfidf_vectorizer.transform(input_df_raw['combined_text']).toarray()
    input_all_features_combined = np.hstack([input_text_features.values, input_tfidf_features])
    input_scaled_features = loaded_scaler.transform(input_all_features_combined)

    try:
        y_pred_proba = model_classifier.predict_proba(input_scaled_features)
        # y_pred_proba her zaman [Normal_Olasılık, Phishing_Olasılık] şeklinde olmalı
        # Positive class (1: Phishing) olasılığı
        phishing_probability = y_pred_proba[0, 1]
        normal_probability = y_pred_proba[0, 0]

        # Tahmin (0.5 eşiği ile)
        y_pred_class = 1 if phishing_probability > 0.5 else 0

        # Tahmine Güven: Modelin verdiği karara ne kadar emin olduğu
        # Eğer Phishing dediyse, phishing olasılığına güvenir.
        # Eğer Normal dediyse, normal olasılığına güvenir.
        confidence_in_prediction = phishing_probability if y_pred_class == 1 else normal_probability

        result = {
            "prediction": "Phishing" if y_pred_class == 1 else "Normal",
            "confidence_in_prediction": float(confidence_in_prediction), # Yeni: Tahmine güven
            "phishing_probability": float(phishing_probability),       # Yeni: Phishing olma olasılığı
            # "confidence" ve "risk_score" yerine bu iki yeni alan kullanıldı.
        }

        print(f"DEBUG: Flask API'den dönen sonuç: {result}")
        return result
    except Exception as e:
        return {"prediction": "Hata", "confidence": 0.0, "risk_score": 1.0, "error": f"Tahmin sırasında hata oluştu: {e}"}

## API Uç Noktaları
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    email_subject = ''
    email_body = ''
    email_sender = ''

    if request.is_json:
        data = request.get_json()
        email_subject = data.get('subject', '')
        email_body = data.get('body', '')
        email_sender = data.get('sender', '')

        # KISITLAMA EKLENDİ: Metin girişi için konu ve/veya gövde metni yeterli uzunlukta olmalı
        MIN_TEXT_LENGTH = 20 # Minimum metin uzunluğu (örneğin 20 karakter)
        if (not email_body or len(email_body) < MIN_TEXT_LENGTH) and \
           (not email_subject or len(email_subject) < MIN_TEXT_LENGTH):
            return jsonify({"error": f"Metin girişi için konu ve/veya gövde en az {MIN_TEXT_LENGTH} karakter uzunluğunda olmalıdır. Lütfen bir e-posta açık olduğundan emin olun."}), 400

        result = predict_email_content(email_subject, email_body, email_sender)
        return jsonify(result)

    elif 'file' in request.files:
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "Dosya seçilmedi."}), 400

        # OCR ile metin çıkarma
        try:
            img = Image.open(BytesIO(file.read()))
            img_cv = np.array(img.convert('RGB'))
            img_gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)

            extracted_text = pytesseract.image_to_string(img_gray, lang='eng+tur')

            # KISITLAMA EKLENDİ: OCR ile çıkarılan metin yeterli uzunlukta olmalı
            MIN_OCR_TEXT_LENGTH = 50 # OCR metni için minimum uzunluk
            if not extracted_text or len(extracted_text) < MIN_OCR_TEXT_LENGTH:
                return jsonify({"error": f"Görselden anlamlı metin çıkarılamadı veya çok kısa (<{MIN_OCR_TEXT_LENGTH} karakter). Daha net bir görsel deneyin."}), 400

            result = predict_email_content(email_subject="Görselden Çıkarıldı", email_body=extracted_text, email_sender="bilinmeyen@gorsel.com")
            return jsonify(result)

        except Exception as e:
            return jsonify({"error": f"Görsel işleme veya OCR hatası: {e}"}), 500

    return jsonify({"error": "Geçersiz istek. Lütfen JSON verisi veya dosya yükleyin."}), 400

def start_ngrok():
    """Ngrok'u ayrı thread'de başlat"""
    time.sleep(2)  # Flask'ın başlamasını bekle
    try:
        # Ngrok auth token'ı ayarla
        ngrok.set_auth_token("2xbU50LBIAiPX9TO6WOShOSLJmv_2X3gwK2Tg4hE18Sp15BTF")

        # Ngrok tünelini başlat
        public_url = ngrok.connect(5000)
        print(f"\n🚀 Flask Uygulaması şu adreste çalışıyor: {public_url}")
        print(f"🔗 Google Extension'da bu URL'i kullan: {public_url}")
        print(f"🧪 Test için: {public_url}/health")

    except Exception as e:
        print(f"❌ Ngrok başlatılırken hata: {e}")



In [ ]:
# =========================================================================
# Flask uygulamasını başlat
# =========================================================================
if __name__ == '__main__':
    print("🔧 Modeller yükleniyor...")
    if not load_models():
        print("❌ Modeller yüklenemedi, uygulama durduruluyor.")
        exit(1)

    print("🌐 Flask sunucusu başlatılıyor...")

    # Ngrok'u ayrı thread'de başlat
    ngrok_thread = threading.Thread(target=start_ngrok)
    ngrok_thread.daemon = True
    ngrok_thread.start()

    # Flask uygulamasını başlat
    app.run(debug=False, host='0.0.0.0', port=5000, threaded=True)

🔧 Modeller yükleniyor...
✅ Tüm modeller ve ön işlemciler başarıyla yüklendi.
🌐 Flask sunucusu başlatılıyor...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



🚀 Flask Uygulaması şu adreste çalışıyor: NgrokTunnel: "https://cc9d-35-204-240-0.ngrok-free.app" -> "http://localhost:5000"
🔗 Google Extension'da bu URL'i kullan: NgrokTunnel: "https://cc9d-35-204-240-0.ngrok-free.app" -> "http://localhost:5000"
🧪 Test için: NgrokTunnel: "https://cc9d-35-204-240-0.ngrok-free.app" -> "http://localhost:5000"/health


INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:45:48] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:45:48] "POST /predict HTTP/1.1" 200 -


DEBUG: Flask API'den dönen sonuç: {'prediction': 'Normal', 'confidence_in_prediction': 0.9999727031306517, 'phishing_probability': 2.729686934831275e-05}


...

DEBUG: Flask API'den dönen sonuç: {'prediction': 'Normal', 'confidence_in_prediction': 0.9733984994589819, 'phishing_probability': 0.026601500541018078}


INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:58:09] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:58:10] "POST /predict HTTP/1.1" 200 -


DEBUG: Flask API'den dönen sonuç: {'prediction': 'Normal', 'confidence_in_prediction': 0.9999727031306517, 'phishing_probability': 2.729686934831275e-05}


INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:58:16] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2025 17:58:17] "POST /predict HTTP/1.1" 200 -


DEBUG: Flask API'den dönen sonuç: {'prediction': 'Normal', 'confidence_in_prediction': 0.9787266697079191, 'phishing_probability': 0.021273330292080853}
